# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
def nhl_cleaner(nhl_df, cities):
    teams_by_ma=cities[["Metropolitan area" ,"Population (2016 est.)[8]","NHL"]]
    teams_by_ma.loc[:, "NHL"] = teams_by_ma["NHL"].replace(r"\[.*\]$","",regex=True)
    nhl_teams_data=nhl_df[nhl_df["year"]==2018]
    nhl_teams_data.loc[:, "team"] = nhl_teams_data["team"].replace(r"\*$", "", regex=True)
    nhl_teams_data=nhl_teams_data[nhl_teams_data["GP"].str.match(r"^[0-9]+$")]
    teams_by_ma = teams_by_ma.dropna(subset=["NHL"])  
    teams_by_ma = teams_by_ma[~teams_by_ma["NHL"].str.contains("—")]
    teams_by_ma = teams_by_ma[teams_by_ma["NHL"].str.strip() != ""]
     

    def name_matcher(nhl_teams,teams_ma=teams_by_ma):
        for index, row in teams_ma.iterrows():
            if re.search(row["NHL"],nhl_teams):
                return row["NHL"]
            else: 
                teams=row["NHL"].split()
                for team in teams:
                    if re.search(team,nhl_teams):
                        return team
        return None

    nhl_teams_data["ma_name"]=nhl_teams_data["team"].apply(name_matcher)

    def area_population_matcher(team_name,population_data=teams_by_ma):
        for index, row in population_data.iterrows():
            if re.search(team_name, row["NHL"],re.IGNORECASE):
                return pd.Series({
                    "pop": row["Population (2016 est.)[8]"],
                    "area": row["Metropolitan area"]
                })

    nhl_teams_data[["Population (2016 est.)[8]", "Metropolitan area"]] = nhl_teams_data["ma_name"].apply(area_population_matcher)      
    nhl_teams_data['W'] = pd.to_numeric(nhl_teams_data['W'], errors='coerce')
    nhl_teams_data['L'] = pd.to_numeric(nhl_teams_data['L'], errors='coerce')
    nhl_teams_data["win_loss_ratio"]= nhl_teams_data['W'] / (nhl_teams_data['L']+nhl_teams_data['W'])
    nhl_win_loss_by_region=nhl_teams_data.groupby('Metropolitan area').agg(
        win_loss_ratio=('win_loss_ratio', 'mean'),
        population=("Population (2016 est.)[8]", 'first')
    )
    return nhl_teams_data, nhl_win_loss_by_region


def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_teams_data, nhl_win_loss_by_region=nhl_cleaner(nhl_df,cities)
    
    population_by_region = nhl_win_loss_by_region["population"].astype(float).values.tolist() # pass in metropolitan area population from cities
    win_loss_by_region = nhl_win_loss_by_region['win_loss_ratio'].astype(float).values.tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()

In [3]:
nhl_correlation()

np.float64(0.012486162921209902)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_cleaner(nba_df, cities):
    nba_teams_by_region=cities[["Metropolitan area" ,"Population (2016 est.)[8]","NBA"]]
    nba_teams_by_region.loc[:, "NBA"] = nba_teams_by_region["NBA"].replace(r"\[.*\]$","",regex=True)
    nba_teams_data=nba_df[nba_df["year"]==2018]
    nba_teams_data.loc[:, "team"] = nba_teams_data["team"].replace(r"\**\s*\([0-9]*\)$", "", regex=True)
    nba_teams_data=nba_teams_data[nba_teams_data["W"].str.match(r"^[0-9]+$")]
    nba_teams_by_region = nba_teams_by_region.dropna(subset=["NBA"])  
    nba_teams_by_region = nba_teams_by_region[~nba_teams_by_region["NBA"].str.contains("—")]
    nba_teams_by_region = nba_teams_by_region[nba_teams_by_region["NBA"].str.strip() != ""]
    
    def name_matcher(nba_teams,teams_ma=nba_teams_by_region):
        for index, row in teams_ma.iterrows():
            if re.search(row["NBA"],nba_teams):
                return row["NBA"]
            else: 
                teams=row["NBA"].split()
                for team in teams:
                    if re.search(team,nba_teams):
                        return team
        return None

    nba_teams_data["ma_name"]=nba_teams_data["team"].apply(name_matcher)

    def area_population_matcher(team_name, population_data=nba_teams_by_region):
        for index, row in population_data.iterrows():
            if re.search(team_name, row["NBA"],re.IGNORECASE):
                return pd.Series({
                    "pop": row["Population (2016 est.)[8]"],
                    "area": row["Metropolitan area"]
                })
    nba_teams_data[["Population (2016 est.)[8]", "Metropolitan area"]] = nba_teams_data["ma_name"].apply(area_population_matcher)      
    nba_teams_data['W'] = pd.to_numeric(nba_teams_data['W'], errors='coerce')
    nba_teams_data['L'] = pd.to_numeric(nba_teams_data['L'], errors='coerce')
    nba_teams_data["win_loss_ratio"]= nba_teams_data['W'] / (nba_teams_data['L']+nba_teams_data['W'] )
    nba_win_loss_by_region=nba_teams_data.groupby('Metropolitan area').agg(
        win_loss_ratio=('win_loss_ratio', 'mean'),
        population=("Population (2016 est.)[8]", 'first')
    )
    return nba_teams_data, nba_win_loss_by_region

def nba_correlation():
    # YOUR CODE HERE
    nba_teams_data, nba_win_loss_by_region= nba_cleaner(nba_df,cities)
    
    population_by_region = nba_win_loss_by_region["population"].astype(float).values.tolist() # pass in metropolitan area population from cities
    win_loss_by_region = nba_win_loss_by_region["win_loss_ratio"].astype(float).values.tolist() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [5]:
nba_correlation()

np.float64(-0.17657160252844617)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_cleaner(mlb_df, cities):
    mlb_teams_by_region=cities[["Metropolitan area" ,"Population (2016 est.)[8]","MLB"]]
    mlb_teams_by_region.loc[:, "MLB"] = mlb_teams_by_region["MLB"].replace(r"\[.*\]$","",regex=True)
    mlb_teams_data=mlb_df[mlb_df["year"]==2018].copy()
    mlb_teams_by_region = mlb_teams_by_region.dropna(subset=["MLB"])  
    mlb_teams_by_region = mlb_teams_by_region[~mlb_teams_by_region["MLB"].str.contains("—")]
    mlb_teams_by_region = mlb_teams_by_region[mlb_teams_by_region["MLB"].str.strip() != ""]



    def name_matcher(mlb_teams,teams_ma=mlb_teams_by_region):
        for index, row in teams_ma.iterrows():
            if re.search(row["MLB"],mlb_teams):
                return row["MLB"]
        
        matching_name=None
        for index, row in teams_ma.iterrows():
            teams_split = row["MLB"].split()
            matches=0
            for word in teams_split:
                if any(word == team_name_word for team_name_word in mlb_teams.split()):
                    matches+=1
                    if matches>1:
                        return matching_name + " " + word
                    else:
                        matching_name = word
        return matching_name

    mlb_teams_data["ma_name"]=mlb_teams_data["team"].apply(name_matcher)
    def area_population_matcher(team_name, population_data=mlb_teams_by_region):
        for index, row in population_data.iterrows():
            if re.search(team_name, row["MLB"],re.IGNORECASE):
                return pd.Series({
                    "pop": row["Population (2016 est.)[8]"],
                    "area": row["Metropolitan area"]
                })
    mlb_teams_data[["Population (2016 est.)[8]", "Metropolitan area"]] = mlb_teams_data["ma_name"].apply(area_population_matcher)      
    mlb_teams_data['W'] = pd.to_numeric(mlb_teams_data['W'], errors='coerce')
    mlb_teams_data['L'] = pd.to_numeric(mlb_teams_data['L'], errors='coerce')
    mlb_teams_data["win_loss_ratio"]= mlb_teams_data['W'] / (mlb_teams_data['L'] + mlb_teams_data['W'])
    mlb_win_loss_by_region=mlb_teams_data.groupby('Metropolitan area').agg(
        win_loss_ratio=('win_loss_ratio', 'mean'),
        population=("Population (2016 est.)[8]", 'first')
    )

    return mlb_teams_data, mlb_win_loss_by_region

def mlb_correlation(): 
    # YOUR CODE HERE
    mlb_teams_data, mlb_win_loss_by_region= mlb_cleaner(mlb_df,cities)
    
    population_by_region = mlb_win_loss_by_region["population"].astype(float).tolist()  # pass in metropolitan area population from cities
    win_loss_by_region = mlb_win_loss_by_region["win_loss_ratio"].astype(float).tolist() # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [13]:
mlb_correlation()

np.float64(0.15027698302669307)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_cleaner(nfl_df, cities):
    nfl_teams_by_region=cities[["Metropolitan area" ,"Population (2016 est.)[8]","NFL"]]
    nfl_teams_by_region.loc[:, "NFL"] = nfl_teams_by_region["NFL"].replace(r"\[.*\]$","",regex=True)
    nfl_teams_data=nfl_df[nfl_df["year"]==2018]
    nfl_teams_by_region = nfl_teams_by_region.dropna(subset=["NFL"]) 
    nfl_teams_data.loc[:, "team"] = nfl_teams_data["team"].replace(r"\*+", "", regex=True)
    nfl_teams_data.loc[:, "team"] = nfl_teams_data["team"].replace(r"\++", "", regex=True)
    nfl_teams_data=nfl_teams_data[nfl_teams_data["W"].str.match(r"^[0-9]+$")] 
    nfl_teams_by_region = nfl_teams_by_region[~nfl_teams_by_region["NFL"].str.contains("—")]
    nfl_teams_by_region = nfl_teams_by_region[nfl_teams_by_region["NFL"].str.strip() != ""]



    def name_matcher(nfl_teams,teams_ma=nfl_teams_by_region):
        for index, row in teams_ma.iterrows():
            if re.search(row["NFL"],nfl_teams):
                return row["NFL"]
        
        matching_name=None
        for index, row in teams_ma.iterrows():
            teams_split = row["NFL"].split()
            matches=0
            for word in teams_split:
                if any(word == team_name_word for team_name_word in nfl_teams.split()):
                    matches+=1
                    if matches>1:
                        return matching_name + " " + word
                    else:
                        matching_name = word
        return matching_name

    nfl_teams_data["ma_name"]=nfl_teams_data["team"].apply(name_matcher)
    def area_population_matcher(team_name, population_data=nfl_teams_by_region):
        for index, row in population_data.iterrows():
            if re.search(team_name, row["NFL"],re.IGNORECASE):
                return pd.Series({
                    "pop": row["Population (2016 est.)[8]"],
                    "area": row["Metropolitan area"]
                })
    nfl_teams_data[["Population (2016 est.)[8]", "Metropolitan area"]] = nfl_teams_data["ma_name"].apply(area_population_matcher)      
    nfl_teams_data['W'] = pd.to_numeric(nfl_teams_data['W'], errors='coerce')
    nfl_teams_data['L'] = pd.to_numeric(nfl_teams_data['L'], errors='coerce')
    nfl_teams_data["win_loss_ratio"]= nfl_teams_data['W'] / (nfl_teams_data['L'] + nfl_teams_data['W'])
    nfl_win_loss_by_region=nfl_teams_data.groupby('Metropolitan area').agg(
        win_loss_ratio=('win_loss_ratio', 'mean'),
        population=("Population (2016 est.)[8]", 'first')
    )

    return nfl_teams_data, nfl_win_loss_by_region




def nfl_correlation(): 
    # YOUR CODE HERE
    
    nfl_teams_data, nfl_win_loss_by_region= nfl_cleaner(nfl_df, cities)
    population_by_region = nfl_win_loss_by_region["population"].astype(float).tolist() # pass in metropolitan area population from cities
    win_loss_by_region = nfl_win_loss_by_region["win_loss_ratio"].astype(float).tolist() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()


In [15]:
nfl_correlation()

np.float64(0.004922112149349462)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
def leagues_cleaner(mlb,nhl,nba,nfl):
    mlb_data,mlb_wl_by_region =mlb_cleaner(mlb,cities)
    nhl_data,nhl_wl_by_region=nhl_cleaner(nhl,cities)
    nba_data,nba_wl_by_region=nba_cleaner(nba,cities)
    nfl_data,nfl_wl_by_region=nfl_cleaner(nfl,cities)
    leagues={"MLB":mlb_wl_by_region,
             "NHL":nhl_wl_by_region,
             "NBA":nba_wl_by_region,
             "NFL":nfl_wl_by_region}
    return leagues
def sports_team_performance():
    # YOUR CODE HERE
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    leagues_wl_by_region=leagues_cleaner(mlb=mlb_df,nhl=nhl_df,nba=nba_df,nfl=nfl_df)
    for league_1 in leagues_wl_by_region.keys():
        league_1_df=leagues_wl_by_region.get(league_1)
        for league_2 in leagues_wl_by_region.keys():
            if league_1 == league_2:
                continue
            league_2_df=leagues_wl_by_region.get(league_2)
            league_merged=pd.merge(league_1_df,league_2_df,how="inner",on="Metropolitan area")
            p_values.loc[league_1,league_2]=stats.ttest_rel(
                league_merged["win_loss_ratio_x"], 
                league_merged["win_loss_ratio_y"])[1]
            
    
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


In [17]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
